In [28]:
import random
import math
import numpy as np

In [27]:
def funcion_p(a, h):
    valor1 = (2 * a + 1)
    valor2 = (a + 1) ** h
    valor3 = 2 * sum(k ** h for k in range(0, a + 1)) 
    return valor1 * valor2 - valor3

def prob_triangular_discreta(a, h, c, d):
    if d < c - a or d > c + a:
        return 0
    numerador = (a + 1) ** h - abs(d - c) ** h
    denominador = funcion_p(a, h)
    prob = numerador / denominador
    return prob

def va_triangular_discreta(a, h, c):
    u = random.random()
    # print(u)
    p_acum = 0
    for d in range(c - a, c + a + 1):
        prob = prob_triangular_discreta(a, h, c, d)
        if u < p_acum:
            return d
        p_acum += prob
    return d

def esperanza_triangular_discreta(a, h, c):
    esperanza = 0
    for d in range(c - a, c + a + 1):
        esperanza += d * prob_triangular_discreta(a, h, c, d)
    return round(esperanza)

In [29]:
def a_value(t):
    if t <= 15:
        return 50
    elif t <= 30:
        return 86
    elif t <= 45:
        return 34
    elif t <= 60:
        return 22
    elif t <= 75:
        return 100
    return 66

def h_value(t):
    if t <= 15:
        return 5
    elif t <= 30:
        return 4
    elif t <= 45:
        return 4
    elif t <= 60:
        return 5
    elif t <= 75:
        return 3
    return 4

def c_value(t):
    if t <= 15:
        return 50
    elif t <= 30:
        return 86
    elif t <= 45:
        return 34
    elif t <= 60:
        return 22
    elif t <= 75:
        return 100
    return 66

def K_value(t):
    if t <= 30:
        return 180
    elif t <= 60:
        return 500
    return 250

def C_value(t):
    if (t % 7 == 0):
        return 18
    return 12

def H_value(t):
    return 1

def Q_value(t):
    return 18

In [ ]:
T = 90
Q = 120

array_size = (Q + 1, T + 1)
matrix = np.zeros(array_size)
matrix_x = np.zeros(array_size)

for j in range(T):
    matrix_x[T][j] = esperanza_triangular_discreta(10, 2, 60)